In [8]:
# WACC+模拟退火
# 最优融资规模模型
import pandas as pd
import numpy as np

# 假设从台账数据中读取的融资需求数据
data = {
    '融资类型': ['短期', '长期'],
    '存量规模': [2000000, 4000000],  # 现有的短期和长期存量规模
    '融资需求': [1000000, 3000000],  # 未来短期和长期的融资需求
    '利率': [0.03, 0.05],  # 短期和长期融资的利率
    '股东权益': [5000000, 6000000],  # 股东权益
    '净利润': [700000, 800000],  # 净利润
    '税率': 0.25  # 企业所得税率
}

df = pd.DataFrame(data)

# 计算WACC
def calculate_wacc(row):
    E = row['股东权益']
    D = row['存量规模']  # 使用存量规模作为债务
    r_e = (row['净利润'] / E)  # ROE可以估算为净利润与股东权益的比率
    r_d = row['利率']
    T = row['税率']
    
    V = E + D  # 资本总额
    
    wacc = (E / V) * r_e + (D / V) * r_d * (1 - T)
    return wacc

# 计算最低融资规模
def calculate_min_funding(df):
    min_funding = {}
    
    for index, row in df.iterrows():
        funding_type = row['融资类型']
        required_return = calculate_wacc(row)  # 获取WACC作为投资的最低回报率
        
        # 通过融资需求和WACC推算最低融资规模
        min_required_funding = row['融资需求'] / required_return  # 以WACC为基准来推算最低融资规模
        
        min_funding[funding_type] = min_required_funding
            
    return min_funding

# 计算最低融资规模
min_funding = calculate_min_funding(df)
total_funding_demand = 100_000_000  # 总融资需求
min_short_term_funding = min_funding['短期']  # 最低短期融资规模
min_long_term_funding = min_funding['长期']   # 最低长期融资规模

# 通过融资规模计算最低比例
min_short_term_ratio = min_short_term_funding / total_funding_demand
min_long_term_ratio = min_long_term_funding / total_funding_demand

# 输出最低融资比例
# print(f"Calculated Minimum Short-term Financing Ratio: {min_short_term_ratio:.2f}")
# print(f"Calculated Minimum Long-term Financing Ratio: {min_long_term_ratio:.2f}")
print(f"WACC计算最低短期融资规模: {min_short_term_funding:.2f}")
print(f"WACC计算最低长期融资规模: {min_long_term_funding:.2f}")
print(f"WACC计算最低短期融资比率: {min_short_term_ratio:.2f}")
print(f"WACC计算最低长期融资比率: {min_long_term_ratio:.2f}")

# 其他假设参数
D_s = 1.0  # 短期流动性需求
D_l = 1.5  # 长期投资需求
R_l = 0.1  # 长期项目回报
c_s = 0.05  # 短期融资成本
c_l = 0.03  # 长期融资成本
risk = 0.2  # 综合风险

# 目标函数
def objective(w_s):
    w_l = 1 - w_s
    liquidity_demand = w_s * D_s + w_l * D_l
    investment_support = w_l * R_l
    total_cost = c_s * w_s + c_l * w_l
    return -(liquidity_demand + investment_support - total_cost)

# 模拟退火算法
def simulated_annealing():
    current_solution = np.random.uniform(min_short_term_ratio, 1 - min_long_term_ratio)
    current_energy = objective(current_solution)
    best_solution = current_solution
    best_energy = current_energy

    T = 1.0  # 初始温度
    T_min = 0.001  # 最小温度
    alpha = 0.9  # 温度衰减率

    while T > T_min:
        # 生成邻域解
        new_solution = current_solution + np.random.uniform(-0.1, 0.1)
        new_solution = np.clip(new_solution, min_short_term_ratio, 1 - min_long_term_ratio)
        new_energy = objective(new_solution)

        # 接受准则
        if new_energy > current_energy or np.random.rand() < np.exp((new_energy - current_energy) / T):
            current_solution = new_solution
            current_energy = new_energy
            
            # 更新最佳解
            if new_energy > best_energy:
                best_solution = new_solution
                best_energy = new_energy

        # 降温
        T *= alpha

    return best_solution, 1 - best_solution  # 返回短期和长期比例

# 运行模拟退火
optimal_short_term_ratio, optimal_long_term_ratio = simulated_annealing()

# 计算基于融资需求的实际金额
optimal_short_term_funding = optimal_short_term_ratio * total_funding_demand
optimal_long_term_funding = optimal_long_term_ratio * total_funding_demand

# 输出结果
# print(f"Optimal Short-term Financing Ratio: {optimal_short_term_ratio:.2f}")
# print(f"Optimal Long-term Financing Ratio: {optimal_long_term_ratio:.2f}")
# print(f"Optimal Short-term Financing Amount: {optimal_short_term_funding:,.2f}")
# print(f"Optimal Long-term Financing Amount: {optimal_long_term_funding:,.2f}")

print(f"模拟退火后：最优短期融资比率: {optimal_short_term_ratio:.2f}")
print(f"模拟退火后：最优长期融资比率: {optimal_long_term_ratio:.2f}")
print(f"模拟退火后：最优短期融资金额: {optimal_short_term_funding:,.2f}")
print(f"模拟退火后：最优长期融资金额: {optimal_long_term_funding:,.2f}")

WACC计算最低短期融资规模: 9395973.15
WACC计算最低长期融资规模: 31578947.37
WACC计算最低短期融资比率: 0.09
WACC计算最低长期融资比率: 0.32
模拟退火后：最优短期融资比率: 0.68
模拟退火后：最优长期融资比率: 0.32
模拟退火后：最优短期融资金额: 68,421,052.63
模拟退火后：最优长期融资金额: 31,578,947.37
